# Acquire Exercises

### *1. Using the code from the lesson as a guide and the REST API from https://python.zgulde.net/api/v1/items .*

In [1]:
# General imports for working on the notebook on time series

import pandas as pd
import requests
import os

In [3]:
# Taking a look at the json file produced from the url

response = requests.get('https://python.zgulde.net/api/v1/items')
response.json()

{'payload': {'items': [{'item_brand': 'Riceland',
    'item_id': 1,
    'item_name': 'Riceland American Jazmine Rice',
    'item_price': 0.84,
    'item_upc12': '35200264013',
    'item_upc14': '35200264013'},
   {'item_brand': 'Caress',
    'item_id': 2,
    'item_name': 'Caress Velvet Bliss Ultra Silkening Beauty Bar - 6 Ct',
    'item_price': 6.44,
    'item_upc12': '11111065925',
    'item_upc14': '11111065925'},
   {'item_brand': 'Earths Best',
    'item_id': 3,
    'item_name': 'Earths Best Organic Fruit Yogurt Smoothie Mixed Berry',
    'item_price': 2.43,
    'item_upc12': '23923330139',
    'item_upc14': '23923330139'},
   {'item_brand': 'Boars Head',
    'item_id': 4,
    'item_name': 'Boars Head Sliced White American Cheese - 120 Ct',
    'item_price': 3.14,
    'item_upc12': '208528800007',
    'item_upc14': '208528800007'},
   {'item_brand': 'Back To Nature',
    'item_id': 5,
    'item_name': 'Back To Nature Gluten Free White Cheddar Rice Thin Crackers',
    'item_price':

### *In the following cells I am trying to understand and remember the content of the current course by going through the code myself*

In [5]:
# Assigning 'response.json()' to a variable and checking out the keys

data = response.json()
data.keys()

dict_keys(['payload', 'status'])

In [7]:
# Checking the keys out

data['status']

'ok'

In [8]:
# Checking the keys out of payload

data['payload'].keys()

dict_keys(['items', 'max_page', 'next_page', 'page', 'previous_page'])

In [9]:
# Displaying items in the data

data['payload']['items']

# Data['payload']['items']['num':'num'] EQUIVALENT because dict

[{'item_brand': 'Riceland',
  'item_id': 1,
  'item_name': 'Riceland American Jazmine Rice',
  'item_price': 0.84,
  'item_upc12': '35200264013',
  'item_upc14': '35200264013'},
 {'item_brand': 'Caress',
  'item_id': 2,
  'item_name': 'Caress Velvet Bliss Ultra Silkening Beauty Bar - 6 Ct',
  'item_price': 6.44,
  'item_upc12': '11111065925',
  'item_upc14': '11111065925'},
 {'item_brand': 'Earths Best',
  'item_id': 3,
  'item_name': 'Earths Best Organic Fruit Yogurt Smoothie Mixed Berry',
  'item_price': 2.43,
  'item_upc12': '23923330139',
  'item_upc14': '23923330139'},
 {'item_brand': 'Boars Head',
  'item_id': 4,
  'item_name': 'Boars Head Sliced White American Cheese - 120 Ct',
  'item_price': 3.14,
  'item_upc12': '208528800007',
  'item_upc14': '208528800007'},
 {'item_brand': 'Back To Nature',
  'item_id': 5,
  'item_name': 'Back To Nature Gluten Free White Cheddar Rice Thin Crackers',
  'item_price': 2.61,
  'item_upc12': '759283100036',
  'item_upc14': '759283100036'},
 {'i

In [ ]:
# Decomposing the code to understand it and reproduce it in a function with a loop

# Setup (DO NOT RUN THIS CELL)
# Compilation of requests to get the data for each page until request hits None
domain = 'https://api.data.codeup.com'
endpoint = '/api/v1/items'
items = []
url = domain + endpoint
response = requests.get(url)
data = response.json()
items.extend(data['payload']['items'])
endpoint = data['payload']['next_page']

# AG ====> 'while true' instead of 'for x in endpoint'
while True:
    url = domain + endpoint
    response = requests.get(url)
    data = response.json()
    items.extend(data['payload']['items'])
    endpoint = data['payload']['next_page']
#     Break the loop if endpoint (for next page) is None
    if endpoint is None:
        break
        
# For printing operation in process (AG)
    print(f'\rGetting page {data["payload"]["page"]} of {data["payload"]["max_page"]}: {url}', end='')

In [22]:
# Reaching out to the API and loop through pages to get dataset
# Transposing setup
endpoint = "items"
host = "https://python.zgulde.net/"
api = "api/v1/"
url = host + api + endpoint
response = requests.get(url)

if response.ok:
    payload = response.json()["payload"]
    contents = payload['items']
    df = pd.DataFrame(contents)
    next_page = payload["next_page"]

   
    while next_page:
        # Append the next_page url piece
        url = host + next_page
        response = requests.get(url)
        payload = response.json()["payload"]
        next_page = payload["next_page"]    
        contents = payload['items']
        df = pd.concat([df, pd.DataFrame(contents)])
        # Drop True because CSV to avoid repeating index and setting all columns as non-index
        df = df.reset_index(drop=True)
items = df
items.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [23]:
items.shape

(50, 6)

In [25]:
# Putting the function together for reproduction

def get_api_data(endpoint):
    '''
    Using a for loop to get each page from
    the data via url until last page.
    Stops the loop when request gets a None
    at the end of the pages
    '''
    
    # API parameters setup and entry check
    if endpoint not in ['items', 'stores', 'sales']:
        return "Entry not for API. Check documentation with: response = requests.get(base_url + /documentation))"
    host = "https://python.zgulde.net/"
    api = "api/v1/"
    url = host + api + endpoint
    response = requests.get(url)
    # Endpoint must only be 'stores', 'items', or 'sales'
    if response.ok:
        payload = response.json()["payload"]
        contents = payload[endpoint]
        df = pd.DataFrame(contents)
        next_page = payload["next_page"]
        # Loading content page after page into a df then printing the process
        while next_page:
            url = host + next_page
            response = requests.get(url)
            payload = response.json()["payload"]
            next_page = payload["next_page"] 
            print(f'\rGetting page {payload["page"]} of {payload["max_page"]}: {url}', end='')      
            contents = payload[endpoint]
            df = pd.concat([df, pd.DataFrame(contents)])
            df = df.reset_index(drop=True) 
    return df

In [27]:
items_df = get_api_data('items')
items_df.head()

Getting page 3 of 3: https://python.zgulde.net//api/v1/items?page=3

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [28]:
# Writing the new df to a local csv file

items_df.to_csv('items_df.csv', index=False)

## *2. Repeating the process for stores*

In [30]:
stores_df = get_api_data('stores')
stores_df.head()

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218


In [31]:
stores_df.shape

(10, 5)

In [32]:
stores_df.to_csv('stores_df.csv', index=False)

## *3. Extract the data for sales (https://python.zgulde.net/api/v1/sales).*

**Repeating the same process as for the others**

In [33]:
sales_df = get_api_data('sales')
sales_df.head()

Getting page 183 of 183: https://python.zgulde.net//api/v1/sales?page=183

,item,sale_amount,sale_date,sale_id,store
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1


In [34]:
sales_df.to_csv('sales_df.csv', index=False)

## *4. Save the data in your files to local csv files*

>### Done along with each function setup

## *5. Combine the data from your three separate dataframes into one large dataframe.*

>### *Displaying the different dataframes to take an ensemble view*

In [ ]:
items_df.head()

In [ ]:
sales_df.head()

In [ ]:
stores_df.head()

In [ ]:
# Making sure another column but the first one isn't the index

sales_df.index

In [ ]:
# Renaming the columns to avoid confusion and enable merger

sales_df.rename(columns={'item': 'item_id', 'store': 'store_id'}, inplace=True)

In [ ]:
# Displaying new sales_df

sales_df.head()

In [ ]:
# Putting the dataframes together

df = pd.merge(sales_df, items_df, how='left', on='item_id')
sales_info = pd.merge(df, stores_df, how='left', on='store_id')

In [ ]:
sales_info.head()

In [ ]:
sales_info.to_csv('sales_info.csv', index=False)

### *Checking the unique store ids and items ids*

In [ ]:
sales_info.store_id.nunique()

In [ ]:
sales_info.item_id.nunique()

In [ ]:
# Getting general information from the df

sales_info.info()

## *6. Acquire the Open Power Systems Data for Germany*

In [ ]:
# Taking a look at the json file produced from the url

response = requests.get('https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv')
response.json()

In [ ]:
# Getting the data from the url

opsd = pd.read_csv('https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv') 

In [ ]:
opsd.head()

In [ ]:
# Writing the acquisition function for Open Power Systems Data for Germany

def get_opsd_germany():
    if os.path.exists('opsd.csv'):
        return pd.read_csv('opsd.csv')
    df = pd.read_csv('https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv')
    df.to_csv('opsd.csv', index=False)
    return df

In [ ]:
data = get_OPS_Germany()

In [ ]:
opsd_df = get_opsd_germany()

In [ ]:
opsd_df.head()

In [ ]:
opsd_df.rename(columns={'Wind+Solar' : 'Wind_and_Solar'}, inplace=True)

In [ ]:
opsd_df.head()

In [ ]:
opsd_df.to_csv('opsd_df.csv', index=False)

### Attempts at more generalizing function

In [ ]:
# Getting api data regardless of endpoint

def get_api_data(endpoint):
    '''
    Using a for loop to get each page from
    the data via url until last page.
    Stops the loop when request gets a None
    at the end of the pages
    '''
    
    # Setup
    if endpoint not in ['items', 'stores', 'sales']:
        return "Entry not for this API. Check documentation with response = requests.get(base_url + /documentation) and print.)"
    
    domain = 'https://python.zgulde.net'
    api = '/api/v1/'
    url = domain + api + endpoint
    response = requests.get(url)
    endpoint = []
    page_range = range(2, max_page + 1)
    for i in endpoint:
        data = response.json()
        payload = response.json()['payload']
        next_page = payload['next_page']
        print(f'\rGetting page {payload["page"]} of {payload["max_page"]}: {url}', end='')      
#         contents = payload[endpoint]
#         # Update the endpoint
        endpoint = next_page + 1
        if endpoint == None:
            break
    return df


# sales.extend(data['payload']['sales'])

In [ ]:
get_api_data('sales')

In [ ]:
def get_api_data(endpoint):
    '''gets data from API with the right endpoint'''
    
    if endpoint not in ['stores', 'items', 'sales']:
        return 'Entry not for this API. Check documentation with response = requests.get(base_url + /documentation) and print.)'
    # Setup
    host = 'https://python.zgulde.net/'
    api = 'api/vl/'
    url = host + api + endpoint
    response = requests.get(url)
    if response.ok:
        data = response.json()
        payload = data['payload']
        # endpoint must only be 'stores', 'items', or 'sales'
        contents = payload[endpoint]
        # Make a dataframe of the contents
        df = pd.DataFrame (contents)
        next_page = payload[ 'next_page' ]
        while next_page:
            url = host + next_page
            response = requests.get(url)
            payload = response.json()['payload']
            next_page = payload[ "next_page" ]
            print (f'\rGetting page {payload["page"]} of {payload["max_page"]}: {url}', end = '')
            contents = payload[endpoint]
            df = pd.concat([df, pd.DataFrame(contents)])
            df = df.reset_index(drop=True)
    return df

In [ ]:
sales_df = get_api_data('sales')

In [ ]:
def get_api_data(endpoint):
    '''gets data from API with the right endpoint'''
    
    if endpoint not in ['stores', 'items', 'sales']:
        return 'Entry not for this API. Check documentation with response = requests.get(base_url + /documentation) and print.)'
    # Setup
    host = 'https://python.zgulde.net/'
    api = 'api/vl/'
    url = host + api + endpoint
    response = requests.get(url)
    if response.ok:
        payload = response.json()['payload']
        # endpoint must only be 'stores', 'items', or 'sales'
        contents = payload[endpoint]
        # Make a dataframe of the contents
        df = pd.DataFrame (contents)
        next_page = payload[ 'next_page' ]
        while next_page:
            url = host + next_page
            response = requests.get(url)
            payload = response.json()['payload']
            next_page = payload[ "next_page" ]
            print (f'\rGetting page {payload["page"]} of {payload["max_page"]}: {url}', end = '')
            contents = payload[endpoint]
            df = pd.concat([df, pd.DataFrame(contents)])
            df = df.reset_index(drop=True)
    return df

In [ ]:
sales_df = get_api_data('sales')